# Building the Sum Calculator Skill
This notebook demonstrates how to use the Cortex Python SDK to build a skill which will invoke an Agent. It will be put into an agent, and when invoked will return the sum of two numbers back to it's calling skill.

In [ ]:
# Common Setup
%run setup.ipynb

We are going to create the sum_returner function. This function will take two numbers as inputs, and return their sum as a json object. This is a very simple skill, which we plan on turning into an Agent so that we can invoke it from another skill. When an input service is created for an agent, a REST API URL is generated, which gives us an access point to invoke it. 

In [ ]:
%%cortex_action --name 'class/calculate-sum-<your_initials>' --function calculate_sum

from cortex import Message

def calculate_sum(params):
    msg = Message(params)
    num1 = msg.payload.get('num1')
    num2 = msg.payload.get('num2')
    
    sum = num1+num2
    
    return Message.with_payload({'Sum': sum}).to_params()

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did above for '--name'
action = cortex.action('class/calculate-sum-<your_initials>')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we will pass two numbers as input, which will be used to calculate the sum and be returned.

In [ ]:
from cortex import Message
rs = action.invoke(Message.with_payload({'num1' : 10, 'num2' : 15}))
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Sum Calculator" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('class/calculate-sum-<your_initials>').title('Sum Calculator-<your_initials>').description('Takes in two numbers and calculates their sum.')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('numbers').title('Input Numbers').parameter(name='num1', type='number').parameter(name='num2', type='number').all_routing(action, 'sum').build()

In the previous step, we referenced an Output called **sum**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('sum').title('Sum').parameter(name='Sum', type='string').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))